In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from seapopym_optimization.cost_function import NoTransportCostFunction, Observation
import pint

from seapopym.configuration.no_transport.parameter import ForcingParameters

from seapopym.configuration.parameters.parameter_forcing import ForcingUnit

from seapopym.standard.units import StandardUnitsLabels

In this example we are creating fake data to test the cost function on many layers.


In [2]:
default_forcing = xr.DataArray(
    data=np.full((6,3,3,3), 1, dtype=float),
    coords={
        "time":pd.date_range("2024-01-01", "2024-01-06"),
        "latitude":[0,1,2],
        "longitude":[0,1,2],
        "depth":[1,2,3]
    }
)
default_forcing.time.attrs = {
    "axis" :"T",
    "long_name" :"Time",
    "standard_name" :"time"
}
default_forcing.latitude.attrs = {
    "axis" :"Y",
    "long_name" :"Latitude",
    "standard_name" :"latitude"
}
default_forcing.longitude.attrs = {
    "axis" :"X",
    "long_name" :"Longitude",
    "standard_name" :"longitude"
}
default_forcing.depth.attrs = {
    "axis" :"Z",
    "long_name" :"Depth",
    "standard_name" :"depth"
}
default_forcing

<xarray.DataArray (time: 6, latitude: 3, longitude: 3, depth: 3)> Size: 1kB
array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
...
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]])
Coordinates:
  * time       (time) datetime64[ns] 48B 2024-01-01 2024-01-02 ... 2024-01-06
  * latitude   (latitude) int64 24B 0 1 2
  * longitude  (longitude) int64 24B 0 1 2
  * depth      (depth) int64 24B 1 2 3

In [3]:
temperature = default_forcing.rename("T")
temperature.attrs = {
    "units": "degree_Celsius"
}

primary_production = default_forcing.rename("NPP").sel(depth=1).drop_vars("depth")
primary_production.attrs = {
    "units": "kilogram / day / meter ** 2"
}

default_obs = default_forcing.copy()
default_obs.attrs = {
    "units" : "kilogram / meter ** 2"
}

In [4]:
temperature

<xarray.DataArray 'T' (time: 6, latitude: 3, longitude: 3, depth: 3)> Size: 1kB
array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
...
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]])
Coordinates:
  * time       (time) datetime64[ns] 48B 2024-01-01 2024-01-02 ... 2024-01-06
  * latitude   (latitude) int64 24B 0 1 2
  * longitude  (longitude) int64 24B 0 1 2
  * depth      (depth) int64 24B 1 2 3
Attributes:
    units:    degree_Celsius

In [5]:
primary_production

<xarray.DataArray 'NPP' (time: 6, latitude: 3, longitude: 3)> Size: 432B
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])
Coordinates:
  * time       (time) datetime64[ns] 48B 2024-01-01 2024-01-02 ... 2024-01-06
  * latitude   (latitude) int64 24B 0 1 2
  * longitude  (longitude) int64 24B 0 1 2
Attributes:
    units:    kilogram / day / meter ** 2

In [6]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=temperature, resolution=1/12),
    primary_production=ForcingUnit(forcing=primary_production, resolution=1/12),
)

In [7]:
observations = Observation(xr.Dataset({"day":default_obs, "night":default_obs}))

In [8]:
fixed_parameters = [
    [None, None, None, None, 1, 1, None],
    [None, None, None, None, 2, 2, None],
    [None, None, None, None, 2, 1, None],
    [None, None, None, None, 3, 3, None],
    [None, None, None, None, 3, 2, None],
    [None, None, None, None, 3, 1, None],
]
cost_function = NoTransportCostFunction(
    nb_parameters=7,
    nb_functional_groups=6,
    fixed_parameters=fixed_parameters,
    forcing_parameters=forcing_parameters,
    observations=[observations],
    groups_name=["D1N1", "D2N2", "D2N1", "D3N3", "D3N2", "D3N1",]
).generate()

In [9]:
cost = cost_function(
    np.tile([11, -0.1, 100, 0.1, 0.2], (1,6)).flatten(),
    
)

2024-10-05 15:20:19,406 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-10-05 15:20:19,419 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-10-05 15:20:19,445 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-10-05 15:20:19,487 :: Seapodym ::  DEBUG ::
|	Direct computation for _average_temperature.

2024-10-05 15:20:19,549 :: Seapodym ::  DEBUG ::
|	Direct computation for _apply_coefficient_to_primary_production_helper.

2024-10-05 15:20:19,558 :: Seapodym ::  DEBUG ::
|	Direct computation for _min_temperature_by_cohort_helper.

2024-10-05 15:20:19,560 :: Seapodym ::  DEBUG ::
|	Direct computation for _mask_temperature_helper.

2024-10-05 15:20:19,582 :: Seapodym ::  DEBUG ::
|	Direct computation for _cell_area_helper.

2024-10-05 15:20:19,610 :: Seapodym ::  DEBUG ::
|	Direct computation for _mortality_field_helper.

2024-10-05 15:20:19,628 :: Seapodym ::  DEBUG ::
|	Direct computation for _productio

In [10]:
cost

(2.0,)